Source: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [7]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.10.1
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [8]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [9]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.',
       b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends

In [10]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

In [11]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 6.53866887e-01,  4.16128516e-01,  3.27893086e-02,
         1.86483651e-01, -3.13032329e-01, -1.68874860e-01,
         4.72123884e-02,  5.54777160e-02, -6.68088198e-01,
         3.75170738e-01,  1.55744404e-01, -2.13741809e-02,
         1.07529223e-01,  1.25133052e-01, -1.01238206e-01,
        -4.39927906e-01, -4.55595165e-01, -7.09834620e-02,
         1.81323647e-01, -6.68889523e-01, -6.25920370e-02,
        -2.76034594e-01,  6.06147110e-01,  2.78416157e-01,
        -2.05840290e-01,  1.19187042e-01, -9.47852433e-01,
         4.38334197e-01,  4.70661044e-01, -2.58364767e-01,
        -4.88546520e-01, -1.67859122e-01,  2.71252751e-01,
        -4.41983014e-01, -2.36122295e-01,  1.09489478e-01,
         4.95491713e-01,  1.47909030e-01,  2.45034203e-01,
        -6.38907194e-01, -8.64069536e-02,  5.26332140e-01,
        -8.07210803e-02,  9.62513313e-02, -3.53074819e-01,
        -6.76786721e-01, -5.46558142e-01, -4.40590918e-01,
       

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)     

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 11s 289ms/step - loss: 0.6425 - accuracy: 0.5582 - val_loss: 0.5799 - val_accuracy: 0.6550
Epoch 2/10
30/30 [==============================] - 9s 283ms/step - loss: 0.5020 - accuracy: 0.7355 - val_loss: 0.4687 - val_accuracy: 0.7419
Epoch 3/10
30/30 [==============================] - 9s 284ms/step - loss: 0.3723 - accuracy: 0.8408 - val_loss: 0.3820 - val_accuracy: 0.8221
Epoch 4/10
30/30 [==============================] - 9s 282ms/step - loss: 0.2752 - accuracy: 0.8944 - val_loss: 0.3361 - val_accuracy: 0.8443
Epoch 5/10
30/30 [==============================] - 10s 312ms/step - loss: 0.2061 - accuracy: 0.9283 - val_loss: 0.3128 - val_accuracy: 0.8620
Epoch 6/10
30/30 [==============================] - 9s 289ms/step - loss: 0.1541 - accuracy: 0.9516 - val_loss: 0.3022 - val_accuracy: 0.8643
Epoch 7/10
30/30 [==============================] - 9s 289ms/step - loss: 0.1151 - accuracy: 0.9685 - val_loss: 0.3004 - val_accuracy: 0.8707
Epoc

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3555 - accuracy: 0.8549 - 3s/epoch - 63ms/step
loss: 0.355
accuracy: 0.855
loss: 0.355
accuracy: 0.855


In [16]:
# What was the point of this? Even without the pretrained model we got similar performance
# pretrained model should've helped a lot but seems like it didn't
# TODO: find a transfer learning example that actually works